In [0]:
dbutils.widgets.text(name="env", defaultValue="", label="Enter environment")
env = dbutils.widgets.get("env")
env

'dev'

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType


In [0]:
%run "./paths"

In [0]:
catalog = f'{env}_catalog'
table = 'state_gdp'
state_gdp_table = f'{catalog}{bronze_tables[table]}'
state_gdp_table

'dev_catalog.bronze.state_gdp_bronze'

('abfss://landing@dlsunitycat.dfs.core.windows.net/',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/bronze',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/silver',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/gold')

In [0]:
state_gdp = spark.table(state_gdp_table)

In [0]:
geo_name_mapping = {
    "United States": "USA",
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

state_gdp = state_gdp.replace(geo_name_mapping, subset=["GeoName"])

state_gdp_avg = (state_gdp
                 .filter(F.col('business_year').between(2018, 2024))
                 .groupBy("GeoName", "business_year")
                 .agg(F.avg("value").cast("DECIMAL(10,2)").alias("average_gdp"))
                 .filter(F.col("GeoName").isin(list(geo_name_mapping.values())))
                 .withColumnRenamed("GeoName", "state_code")
                 .withColumn("date_ingested", F.current_timestamp())
)

state_gdp_avg.write.format("delta").mode("overwrite").save(gold_paths[table])
